In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    DDD_Train_Decile3 = pd.read_pickle("DDD_Train_Decile3.pkl")
    DDD_Train_Targets_1D = pd.read_pickle("DDD_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("DDD_Test_Decile3.pkl")
    y_test = pd.read_pickle("DDD_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(DDD_Train_Decile3, DDD_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM DDD Decile 3')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

Epoch 38/100
2264/2264 [==============================] - 2s 970us/step - loss: 0.4853 - acc: 0.7937 - val_loss: 0.4755 - val_acc: 0.7884
Epoch 39/100
2264/2264 [==============================] - 2s 978us/step - loss: 0.4790 - acc: 0.7902 - val_loss: 0.4832 - val_acc: 0.7848
Epoch 40/100
2264/2264 [==============================] - 2s 960us/step - loss: 0.4803 - acc: 0.7973 - val_loss: 0.4771 - val_acc: 0.7760
Epoch 41/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4770 - acc: 0.7924 - val_loss: 0.4691 - val_acc: 0.7901
Epoch 42/100
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4778 - acc: 0.7867 - val_loss: 0.4705 - val_acc: 0.7866
Epoch 43/100
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4785 - acc: 0.7946 - val_loss: 0.4706 - val_acc: 0.7884
Epoch 44/100
2264/2264 [==============================] - 2s 994us/step - loss: 0.4731 - acc: 0.7933 - val_loss: 0.4632 - val_acc: 0.7954
Epoch 45/100
2264/2264 [================

2264/2264 [==============================] - 2s 926us/step - loss: 0.4374 - acc: 0.8021 - val_loss: 0.4349 - val_acc: 0.8025
Epoch 55/150
2264/2264 [==============================] - 2s 916us/step - loss: 0.4354 - acc: 0.8012 - val_loss: 0.4348 - val_acc: 0.8025
Epoch 56/150
2264/2264 [==============================] - 2s 931us/step - loss: 0.4348 - acc: 0.8021 - val_loss: 0.4347 - val_acc: 0.8025
Epoch 57/150
2264/2264 [==============================] - 2s 929us/step - loss: 0.4308 - acc: 0.8061 - val_loss: 0.4346 - val_acc: 0.8025
Epoch 58/150
2264/2264 [==============================] - 2s 919us/step - loss: 0.4350 - acc: 0.8034 - val_loss: 0.4344 - val_acc: 0.8042
Epoch 59/150
2264/2264 [==============================] - 2s 937us/step - loss: 0.4377 - acc: 0.8043 - val_loss: 0.4340 - val_acc: 0.8095
Epoch 60/150
2264/2264 [==============================] - 2s 919us/step - loss: 0.4355 - acc: 0.8017 - val_loss: 0.4336 - val_acc: 0.8113
Epoch 61/150
2264/2264 [=======================

2264/2264 [==============================] - 2s 1ms/step - loss: 0.4666 - acc: 0.7942 - val_loss: 0.4664 - val_acc: 0.7884
Epoch 23/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4669 - acc: 0.7951 - val_loss: 0.4613 - val_acc: 0.7866
Epoch 24/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4689 - acc: 0.7928 - val_loss: 0.4633 - val_acc: 0.7901
Epoch 25/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4639 - acc: 0.7911 - val_loss: 0.4626 - val_acc: 0.7848
Epoch 26/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4607 - acc: 0.7902 - val_loss: 0.4670 - val_acc: 0.7848
Epoch 27/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4588 - acc: 0.7937 - val_loss: 0.4634 - val_acc: 0.7901
Epoch 28/150
2264/2264 [==============================] - 3s 1ms/step - loss: 0.4613 - acc: 0.7937 - val_loss: 0.4557 - val_acc: 0.7901
Epoch 29/150
2264/2264 [==============================] - 3s 

Epoch 81/150
2264/2264 [==============================] - 2s 980us/step - loss: 0.4391 - acc: 0.8079 - val_loss: 0.4371 - val_acc: 0.8060
Epoch 82/150
2264/2264 [==============================] - 2s 968us/step - loss: 0.4410 - acc: 0.8021 - val_loss: 0.4371 - val_acc: 0.8060
Epoch 83/150
2264/2264 [==============================] - ETA: 0s - loss: 0.4358 - acc: 0.803 - 2s 1ms/step - loss: 0.4352 - acc: 0.8034 - val_loss: 0.4374 - val_acc: 0.8025
Epoch 84/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4393 - acc: 0.8052 - val_loss: 0.4375 - val_acc: 0.8025
Epoch 85/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4385 - acc: 0.8008 - val_loss: 0.4374 - val_acc: 0.8025
Epoch 86/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4441 - acc: 0.8052 - val_loss: 0.4373 - val_acc: 0.8025
Epoch 87/150
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4417 - acc: 0.8061 - val_loss: 0.4375 - val_acc: 0.8042
Epoch 

...

Epoch 137/200
2264/2264 [==============================] - 2s 946us/step - loss: 0.4326 - acc: 0.8083 - val_loss: 0.4278 - val_acc: 0.8078
Epoch 138/200
2264/2264 [==============================] - 2s 939us/step - loss: 0.4282 - acc: 0.8043 - val_loss: 0.4278 - val_acc: 0.8078
Epoch 139/200
2264/2264 [==============================] - 2s 926us/step - loss: 0.4248 - acc: 0.8070 - val_loss: 0.4280 - val_acc: 0.8060
Epoch 140/200
2264/2264 [==============================] - 2s 953us/step - loss: 0.4266 - acc: 0.8096 - val_loss: 0.4280 - val_acc: 0.8095
Epoch 141/200
2264/2264 [==============================] - 2s 967us/step - loss: 0.4346 - acc: 0.8052 - val_loss: 0.4280 - val_acc: 0.8095
Epoch 142/200
2264/2264 [==============================] - 2s 952us/step - loss: 0.4309 - acc: 0.8074 - val_loss: 0.4280 - val_acc: 0.8060
Epoch 143/200
2264/2264 [==============================] - 2s 929us/step - loss: 0.4345 - acc: 0.8043 - val_loss: 0.4278 - val_acc: 0.8078
Epoch 144/200
2264/2264 [==

2264/2264 [==============================] - 2s 984us/step - loss: 0.4696 - acc: 0.7955 - val_loss: 0.4661 - val_acc: 0.7848
Epoch 56/100
2264/2264 [==============================] - 2s 972us/step - loss: 0.4712 - acc: 0.7924 - val_loss: 0.4654 - val_acc: 0.7831
Epoch 57/100
2264/2264 [==============================] - 2s 973us/step - loss: 0.4641 - acc: 0.7862 - val_loss: 0.4659 - val_acc: 0.7848
Epoch 58/100
2264/2264 [==============================] - 2s 959us/step - loss: 0.4658 - acc: 0.7889 - val_loss: 0.4651 - val_acc: 0.7884
Epoch 59/100
2264/2264 [==============================] - 2s 954us/step - loss: 0.4670 - acc: 0.7928 - val_loss: 0.4668 - val_acc: 0.7795
Epoch 60/100
2264/2264 [==============================] - 2s 954us/step - loss: 0.4676 - acc: 0.7920 - val_loss: 0.4651 - val_acc: 0.7813

Epoch 00060: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 61/100
2264/2264 [==============================] - 2s 999us/step - loss: 0.4633 - acc: 0.7924 - val_loss: 0.4645 

2264/2264 [==============================] - 2s 1ms/step - loss: 0.4348 - acc: 0.8092 - val_loss: 0.4331 - val_acc: 0.8078
Epoch 73/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4363 - acc: 0.8008 - val_loss: 0.4330 - val_acc: 0.8078
Epoch 74/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4338 - acc: 0.8087 - val_loss: 0.4330 - val_acc: 0.8078
Epoch 75/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4311 - acc: 0.8074 - val_loss: 0.4331 - val_acc: 0.8078
Epoch 76/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4344 - acc: 0.8061 - val_loss: 0.4330 - val_acc: 0.8078
Epoch 77/200
2264/2264 [==============================] - ETA: 0s - loss: 0.4321 - acc: 0.806 - 2s 1ms/step - loss: 0.4333 - acc: 0.8048 - val_loss: 0.4329 - val_acc: 0.8095
Epoch 78/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4414 - acc: 0.8030 - val_loss: 0.4330 - val_acc: 0.8078
Epoch 79/200
2264/2264 

Epoch 191/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4334 - acc: 0.8052 - val_loss: 0.4323 - val_acc: 0.8078
Epoch 192/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4339 - acc: 0.8074 - val_loss: 0.4322 - val_acc: 0.8078
Epoch 193/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4346 - acc: 0.8083 - val_loss: 0.4322 - val_acc: 0.8078
Epoch 194/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4346 - acc: 0.8070 - val_loss: 0.4323 - val_acc: 0.8078
Epoch 195/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4330 - acc: 0.8052 - val_loss: 0.4322 - val_acc: 0.8078
Epoch 196/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4344 - acc: 0.8061 - val_loss: 0.4322 - val_acc: 0.8078
Epoch 197/200
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4338 - acc: 0.8048 - val_loss: 0.4322 - val_acc: 0.8078
Epoch 198/200
2264/2264 [================

Epoch 50/50
2264/2264 [==============================] - 2s 1ms/step - loss: 0.4329 - acc: 0.8039 - val_loss: 0.4339 - val_acc: 0.8060
Test accuracy: 0.8059964731887534
Evalutation of best performing model:
1150/1150 [==============================] - 0s 276us/step
[0.5211195180727088, 0.7713043477224267]
Best performing model chosen hyper-parameters:
{'epochs': 2, 'lr': 1}
Confusion Matrix for LSTM predictions:
[[104 254]
 [  9 783]]
Classification Report for LSTM predictions:
-------------------------------------------
              precision    recall  f1-score   support

           0       0.92      0.29      0.44       358
           1       0.76      0.99      0.86       792

   micro avg       0.77      0.77      0.77      1150
   macro avg       0.84      0.64      0.65      1150
weighted avg       0.81      0.77      0.73      1150

Accuracy score: 0.7713043478260869


In [ ]:
0.86434